In [ ]:
import pymc3 as pm
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-darkgrid')

In [ ]:
iris = sns.load_dataset('iris')
iris.head()

In [ ]:
iris.shape

In [ ]:
sns.pairplot(iris, hue='species', diag_kind='kde')

plt.figure()

# 正規分布モデル

## 正規分布を仮定

In [ ]:
iris_1 = iris[iris["species"]=="versicolor"].sepal_length
iris_1.head()

In [ ]:
sns.kdeplot(iris_1)

In [ ]:
with pm.Model() as model_1A:
    mu = pm.Uniform('mu', 5.0, 7.0)
    sigma = pm.HalfNormal('sigma', sd=1.0)
    y = pm.Normal('y', mu=mu, sd=sigma, observed=iris_1)
    trace = pm.sample(2000, njobs=4)

trace = trace[200:]
pm.traceplot(trace)

plt.figure()

In [ ]:
#pm.model_to_graphviz(model_1A)

In [ ]:
pm.summary(trace)

In [ ]:
samples=100

ppc = pm.sample_posterior_predictive(trace, samples=samples, model=model_1A)
ppc

In [ ]:
for i in range(samples):
    sns.kdeplot(ppc['y'][i], color='r', alpha=0.05)
    
sns.kdeplot(iris_1)
plt.figure();

幅を広く取りすぎている？

## t分布を仮定

In [ ]:
#t分布とは
x = np.arange(-5,5,0.01)
y1 = stats.t.pdf(x=x, df=1)
y3 = stats.t.pdf(x=x, df=3)
y10 = stats.t.pdf(x=x, df=10)
norm = stats.norm.pdf(x=x,loc=0,scale=1)

plt.plot(x,y1, label='ν=1')
plt.plot(x,y3, label='ν=3')
plt.plot(x,y10, label='ν=10')
plt.plot(x,norm, label='norm')

plt.legend()

In [ ]:
with pm.Model() as model_1B:
    mu = pm.Uniform('mu', 5.0, 7.0)
    sigma = pm.HalfNormal('sigma', sd=1.0)
    nu = pm.Exponential('nu', 1/30)
    y = pm.StudentT('y', mu=mu, sd=sigma, nu=nu, observed=iris_1)
    trace = pm.sample(2000, njobs=4)

trace = trace[200:]
pm.traceplot(trace)

plt.figure();

In [ ]:
#pm.model_to_graphviz(model_1B)

In [ ]:
pm.summary(trace)

In [ ]:
samples=100

ppc = pm.sample_posterior_predictive(trace, samples=samples, model=model_1B)

for i in range(samples):
    sns.kdeplot(ppc['y'][i], color='r', alpha=0.05)
    
sns.kdeplot(iris_1)
plt.figure()

# 線形回帰モデル

In [ ]:
iris_2X = iris['petal_length']
iris_2Y = iris['petal_width']

plt.plot(iris_2X, iris_2Y, 'b.')

In [ ]:
with pm.Model() as model_2A:
    a = pm.Normal('a', mu=0, sd=10)
    b = pm.Normal('b', mu=0, sd=1)
    epsilon = pm.HalfCauchy('epsilon', 5)

    mu = pm.Deterministic('mu', a + b * iris_2X)
    y_pred = pm.Normal('y_pred', mu=mu, sd=epsilon, observed=iris_2Y)
    
    start = pm.find_MAP()
    step = pm.Metropolis()

    trace = pm.sample(2000, step, start, njobs=4)
    
trace = trace[200:]
pm.traceplot(trace)

plt.figure()

In [ ]:
#pm.model_to_graphviz(model_2A)

パラメータ推定がうまくいっていない

強い自己相関があるため

In [ ]:
var = ['a', 'b', 'epsilon']
pm.autocorrplot(trace, var)

plt.figure()

In [ ]:
sns.kdeplot(trace['a'], trace['b'])

改善策：サンプリング方法を変える（NUTS，デフォルト設定）

In [ ]:
with pm.Model() as model_2B:
    a = pm.Normal('a', mu=0, sd=10)
    b = pm.Normal('b', mu=0, sd=1)
    epsilon = pm.HalfCauchy('epsilon', 5)

    mu = pm.Deterministic('mu', a + b *  iris_2X)
    y_pred = pm.Normal('y_pred', mu=mu, sd=epsilon, observed=iris_2Y)

    trace = pm.sample(2000, njobs=4)
    
trace = trace[200:]
pm.traceplot(trace)

plt.figure()

In [ ]:
plt.plot(iris_2X, iris_2Y, 'b.')

a_m = trace['a'].mean()
b_m = trace['b'].mean()
plt.plot(iris_2X, a_m + b_m * iris_2X, c='k',
         label="y={:.2f}*x+{:.2f}".format(a_m, b_m))

plt.legend();

In [ ]:
samples = 1000

ppc = pm.sample_posterior_predictive(trace, samples=samples, model=model_2B)
ppc

In [ ]:
idx = np.argsort(iris_2X)
x_ord = iris_2X[idx]

plt.plot(iris_2X, iris_2Y, "b.")

plt.plot(iris_2X, a_m + b_m * iris_2X, c='k',
         label="y={:.2f}*x+{:.2f}".format(a_m, b_m))

sig0 = pm.hpd(ppc["y_pred"], alpha=0.5)[idx]  # 50%HPD
sig1 = pm.hpd(ppc["y_pred"], alpha=0.05)[idx]  # 95%HPD
plt.fill_between(x_ord, sig0[:, 0], sig0[:, 1], color="gray", alpha=1)
plt.fill_between(x_ord, sig1[:, 0], sig1[:, 1], color="gray", alpha=0.5)

plt.legend();

# ロジスティック回帰モデル

## 1変数

In [ ]:
sns.stripplot(x='species', y='sepal_length', data=iris)

In [ ]:
df = iris.query("species == ('setosa', 'versicolor')")
iris_3Y = pd.Categorical(df['species']).codes
x_n = 'sepal_length' 
iris_3X = df[x_n].values

plt.plot(iris_3X, iris_3Y, 'b.')

In [ ]:
with pm.Model() as model_3A:
    a = pm.Normal('a', mu=0, sd=10)
    b = pm.Normal('b', mu=0, sd=10)
  
    mu = a + pm.math.dot(iris_3X, b)
    theta = pm.Deterministic('theta', 1 / (1 + pm.math.exp(-mu)))
    bd = pm.Deterministic('bd', -a/b)
    yl = pm.Bernoulli('yl', p=theta, observed=iris_3Y)

    trace = pm.sample(2000, njobs=4)

chain = trace[200:]
varnames = ['a', 'b', 'bd']
pm.traceplot(chain, varnames)

plt.figure()

In [ ]:
#pm.model_to_graphviz(model_3A)

In [ ]:
theta = chain['theta'].mean(axis=0)
idx = np.argsort(iris_3X)
plt.plot(iris_3X[idx], theta[idx], color='b', lw=3)
plt.axvline(chain['bd'].mean(), ymax=1, color='r')
bd_hpd = pm.hpd(chain['bd'])
plt.fill_betweenx([0, 1], bd_hpd[0], bd_hpd[1], color='r', alpha=0.5)

plt.plot(iris_3X, iris_3Y, 'o', color='k')
theta_hpd = pm.hpd(chain['theta'])[idx]
plt.fill_between(iris_3X[idx], theta_hpd[:, 0],
                 theta_hpd[:, 1], color='b', alpha=0.5)

plt.xlabel(x_n, fontsize=16)
plt.ylabel(r'$\theta$', rotation=0, fontsize=16)

plt.figure()

## 2変数

In [ ]:
sns.pairplot(iris, hue='species', vars=['sepal_length', 'sepal_width'])

In [ ]:
df = iris.query("species == ('setosa', 'versicolor')")
iris_3Y2 = pd.Categorical(df['species']).codes
x_n = ['sepal_length', 'sepal_width']
iris_3X2 = df[x_n].values

In [ ]:
with pm.Model() as model_3B:
    a = pm.Normal('a', mu=0, sd=10)
    b = pm.Normal('b', mu=0, sd=2, shape=len(x_n))

    mu = a + pm.math.dot(iris_3X2, b)
    theta = pm.Deterministic('theta', 1 / (1 + pm.math.exp(-mu)))
    bd = pm.Deterministic('bd', -a/b[1]-b[0]/b[1]*iris_3X2[:, 0])
    yl = pm.Bernoulli('yl', p=theta, observed=iris_3Y2)

    trace = pm.sample(2000, njobs=4)

chain = trace[200:]
varnames = ['a', 'b', 'bd']
pm.traceplot(chain, varnames)

plt.figure()

In [ ]:
#pm.model_to_graphviz(model_3B)

In [ ]:
idx = np.argsort(iris_3X2[:,0])
ld = chain['bd'].mean(0)[idx]

plt.scatter(iris_3X2[:,0], iris_3X2[:,1], c=iris_3Y, cmap='viridis')
plt.plot(iris_3X2[:,0][idx], ld, color='r');

ld_hpd = pm.hpd(chain['bd'])[idx]
plt.fill_between(iris_3X2[:,0][idx], ld_hpd[:,0], ld_hpd[:,1], color='r', alpha=0.5);

plt.figure()

# 階層モデル

## グループ間の比較

In [ ]:
sns.stripplot(x='species', y='petal_width', data=iris)

In [ ]:
iris_4A = iris['petal_width'].values
idx = pd.Categorical(iris['species']).codes

In [ ]:
with pm.Model() as model_4A:
    means = pm.Normal('means', mu=0, sd=10, shape=len(set(idx)))
    sds = pm.HalfNormal('sds', sd=10, shape=len(set(idx)))

    y = pm.Normal('y', mu=means[idx], sd=sds[idx], observed=iris_4A)

    trace = pm.sample(2000, njobs=4)
chain = trace[200::]
pm.traceplot(chain)

plt.figure()

In [ ]:
#pm.model_to_graphviz(model_4A)

In [ ]:
pm.summary(chain, varnames=['means'])

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.subplots(3, 1, sharex=True)

pm.plot_posterior(chain, varnames=['means'], kde_plot=True, ax=ax);

## 階層モデル

In [ ]:
with pm.Model() as model_4B:
    #階層事前分布
    amean = pm.Normal('am', mu=0, sd=10)
    asd = pm.HalfNormal('asd', sd=10)
    ssd = pm.HalfNormal('ssd', sd=10)
    #事前分布
    means = pm.Normal('means', mu=amean, sd=asd, shape=len(set(idx)))
    sds = pm.HalfNormal('sds', sd=ssd, shape=len(set(idx)))
    #尤度
    y = pm.Normal('y', mu=means[idx], sd=sds[idx], observed=iris_4A)

    trace = pm.sample(2000, njobs=4)
chain = trace[200::]
pm.traceplot(chain)

plt.figure()

In [ ]:
#pm.model_to_graphviz(model_4B)

In [ ]:
pm.summary(chain, varnames=['means'])

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.subplots(3, 1, sharex=True)

pm.plot_posterior(chain, varnames=['means'], kde_plot=True, ax=ax);